In [7]:
import simpy
from tkinter import *
import tkinter as tk
from tkinter import messagebox
from PIL import ImageTk, Image
import qrcode

class Cafeteria:
    def __init__(self, env, food_items):
        self.env = env
        self.food_items = food_items
        self.receive_counter = simpy.Resource(env, capacity=1)
        self.order_queue = []
        self.wait_times = []

    def order_food(self, customer, food_items):
        food_str = ', '.join(food_items)
        # print(f"{customer} ordered: {food_str}")
        # yield self.env.timeout(1)

    def receive_food(self, customer, food_items, receive_window, order_number):
        with self.receive_counter.request() as request:
            yield request
            print(f"{customer} receiving food")
            for food_item in food_items:
                yield self.env.timeout(1)
                self.food_items[food_item]['container'].get(1)
                receive_window.insert(tk.END, f"{food_item}\n")
            yield self.env.timeout(5)
            self.wait_times.append((order_number, self.env.now))
            print(f"{customer} received food")

    def take_order(self, receive_window, order_number, customer_name):
        if not self.order_queue:
            receive_window.delete('1.0', tk.END)
            receive_window.insert(tk.END, "Thank you for choosing us. Enjoy your order\n")
            return None
        order_time, order_items, order_customer_name = self.order_queue.pop(0)
        if self.env.now - order_time > 120:
            return None
        remaining_time = 120 - (self.env.now - order_time)
        receive_window.delete('1.0', tk.END)
        receive_window.insert(tk.END, f"Customer: {customer_name}\n")
        receive_window.insert(tk.END, f"Order: {', '.join([f'{food_item}: {order_items.count(food_item)}' for food_item in set(order_items)])}\n")
        receive_window.insert(tk.END, f"Your order will be ready in {remaining_time} seconds")
        return order_number

    def customer(self, customer, food_items, order_number):
        order_time = self.env.now
        self.order_queue.append((order_time, food_items, customer))
        yield self.env.process(self.order_food(customer, food_items))
        receive_window = tk.Text(width=25, height=5)
        receive_window.pack()
        order_number = yield self.env.process(self.receive_food(customer, food_items, receive_window, order_number))
        receive_window.insert(tk.END, f"{customer} received food")
        receive_window.config(state='disabled')

def run_simulation(env, num_customers, food_items):
    root = tk.Tk()
    root.geometry('680x1080+0+0')
    root.iconbitmap('C:\\Users\\death\\Desktop\\projects\\sem projects\\fo.ico')
    image_file = Image.open("C:\\Users\\death\\Desktop\\projects\\sem projects\\background_image.png")
    image = image_file.resize((680,1080), Image.LANCZOS)
    tk_image = ImageTk.PhotoImage(image)
    background_label = tk.Label(root, image=tk_image)
    background_label.place(x=0, y=0, relwidth=1, relheight=1)
    def show_order_window():
        order_window = tk.Toplevel(root)
        order_window.geometry('680x500+0+0')
        order_window.configure(background="Lavender Blush")
        order_window.iconbitmap('C:\\Users\\death\\Desktop\\projects\\sem projects\\fo.ico')

        name_label = tk.Label(order_window, text='Enter your name:')
        name_label.pack(padx=10, pady=5)
        name_entry = tk.Entry(order_window,justify="center")
        name_entry.pack(padx=10, pady=5)
        order_items = []
        for food_item in food_items:
            if food_items[food_item]['container'].level > 0:
                food_item_frame = tk.Frame(order_window)
                food_item_frame.pack(fill='x', padx=10, pady=5)
                tk.Label(food_item_frame, text=food_item, width=10).pack(side='left')
                tk.Label(food_item_frame, text=f"Left: {food_items[food_item]['container'].level}", width=8).pack(side='right')
                tk.Button(food_item_frame, text='Add to Order', width=10, command=lambda food_item=food_item: add_item_to_order(food_item)).pack(side='right')
            else:
                tk.Label(order_window, text=f"{food_item} is out of stock", fg='red').pack(padx=10, pady=5)
        order_label = tk.Label(order_window, text='')
        order_label.pack()
        def add_item_to_order(item):
            if food_items[item]['container'].level > 0:
                order_items.append(item)
                order_str = ', '.join([f"{food_item}: {order_items.count(food_item)} (₹{food_items[food_item]['price']})" for food_item in set(order_items)])
                order_label.config(text=order_str)
                food_items[item]['container'].get(1)
                for widget in order_window.winfo_children():
                    if isinstance(widget, tk.Frame):
                        if widget.winfo_children()[0].cget('text') == item:
                            widget.winfo_children()[1].config(text=f"Left: {food_items[item]['container'].level}")
            else:
                tk.messagebox.showerror("Out of Stock", f"{item} is out of stock")
        def do_payment(item):
            total_cost = sum([food_items[item]['price'] for item in order_items])
            payment_window = tk.Toplevel(order_window)
            payment_window.iconbitmap('C:\\Users\\death\\Desktop\\projects\\sem projects\\fo.ico')
            payment_window.geometry('300x400')
            payment_window.configure(background="Lavender Blush")
            payment_label = tk.Label(payment_window, text=f"Total amount: ₹{total_cost}")
            payment_label.pack(padx=10, pady=10)
            qr_code = qrcode.make(f"Total amount: ₹{total_cost}")
            qr_code_image = qr_code.resize((200, 200))
            qr_code_photo = ImageTk.PhotoImage(qr_code_image)
            qr_code_label = tk.Label(payment_window, image=qr_code_photo)
            qr_code_label.image = qr_code_photo
            qr_code_label.pack(padx=10, pady=10)
            def process_payment():
                tk.messagebox.showinfo("Payment Done", "Payment has been successfully processed.")
                payment_window.destroy()
            tk.Button(payment_window, text='Payment Done', command=process_payment).pack(padx=10, pady=10)
        def place_order():
            if not order_items:
                tk.messagebox.showerror("Error", "Please select at least one food item")
                return
            customer_name = name_entry.get()
            if not customer_name:
                tk.messagebox.showerror("Error", "Please enter your name")
                return
            total_cost = sum([food_items[item]['price'] for item in order_items])
            payment_window = tk.Toplevel(order_window)
            payment_window.geometry('300x400')
            payment_window.iconbitmap('C:\\Users\\death\\Desktop\\projects\\sem projects\\fo.ico')
            payment_window.configure(background="Lavender Blush")
            payment_label = tk.Label(payment_window, text=f"Total amount: ₹{total_cost}")
            payment_label.pack(padx=10, pady=10)
            qr_code = qrcode.make(f"Total amount: ₹{total_cost}")
            qr_code_image = qr_code.resize((200, 200))
            qr_code_photo = ImageTk.PhotoImage(qr_code_image)
            qr_code_label = tk.Label(payment_window, image=qr_code_photo)
            qr_code_label.image = qr_code_photo
            qr_code_label.pack(padx=10, pady=10)
            def process_payment():
                tk.messagebox.showinfo("Payment Done", "Payment has been successfully processed.")
                payment_window.destroy()
                order_number = len(cafeteria.wait_times) + 1
                env.process(cafeteria.customer(f'{customer_name} (Order {order_number})', order_items, order_number))
                order_window.destroy()
                show_receive_window(order_items, order_number, customer_name)
            tk.Button(payment_window, text='Payment Done', command=process_payment).pack(padx=10, pady=10)
        def cancel_order():
            for item in order_items:
                food_items[item]['container'].put(1)
                for widget in order_window.winfo_children():
                    if isinstance(widget, tk.Frame):
                        if widget.winfo_children()[0].cget('text') == item:
                            widget.winfo_children()[1].config(text=f"Left: {food_items[item]['container'].level}")
            order_items.clear()
            order_label.config(text='')
        but=tk.Button(order_window, text="Place Order", command=place_order)
        but.pack()
        tk.Button(order_window, text='Cancel Order', command=cancel_order).pack()
    def show_receive_window(order_items, order_number, customer_name):
        for child in root.winfo_children():
            if isinstance(child, tk.Toplevel) and child.title() == f"Order-{order_number}":
                receive_window = child
                break
        else:
            receive_window = tk.Toplevel(root)
            receive_window.geometry('600x1080+680+0')
            receive_window.iconbitmap('C:\\Users\\death\\Desktop\\projects\\sem projects\\fo.ico')
            receive_window.configure(background="Lavender Blush")
            receive_window.title(f"Order-{order_number}")
        
        tk.Button(receive_window, text='Take Order', command=lambda: take_order_and_close(receive_window, order_number, customer_name)).pack()
        remaining_time = 120
        receive_text = tk.Text(receive_window, height=5, width=25)
        receive_text.pack()
        receive_text.insert(tk.END, f"Customer: {customer_name}\n")
        receive_text.insert(tk.END, f"Order: {', '.join([f'{food_item}: {order_items.count(food_item)}' for food_item in set(order_items)])}\n")
        receive_label = tk.Label(receive_window, text=f"Your order will be ready in {remaining_time} seconds")
        receive_label.pack()

        def countdown():
            nonlocal remaining_time
            if remaining_time > 0:
                remaining_time -= 1
                receive_label.config(text=f"Your order will be ready in {remaining_time} seconds")
                receive_window.after(1000, countdown)
            else:
                receive_label.config(text="Your order is ready to take")

        countdown()

        def take_order_and_close(window, order_number, customer_name):
            order_number = cafeteria.take_order(receive_text, order_number, customer_name)
            if order_number is not None:
                window.title(f"Order-{order_number}")

    image = tk.PhotoImage(file='C:\\Users\\death\\Desktop\\projects\\sem projects\\tap.png')
    cafeteria = Cafeteria(env, food_items)
    button=tk.Button(root,image=image,command=show_order_window)
    button.configure(width=150, height=100)
    button.place(x=280,y=850)
    env.run(until=30)
    root.mainloop()

if __name__ == '__main__':
    env = simpy.Environment()
    food_items = {
        'PIZZA': {'container': simpy.Container(env, capacity=10, init=10), 'price': 150},
        'BURGER': {'container': simpy.Container(env, capacity=20, init=20), 'price': 50},
        'SANDWICH': {'container': simpy.Container(env, capacity=30, init=30), 'price': 30},
        'PASTA': {'container': simpy.Container(env, capacity=15, init=15), 'price': 15},
        'NOODLES': {'container': simpy.Container(env, capacity=25, init=25), 'price': 15}
    }
    run_simulation(env, 5, food_items)
